<a href="https://www.kaggle.com/code/luogin/the-best-gpt-sovits?scriptVersionId=191589490" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## 电脑必选Edge或者Chrome
## Please use Edge or Chrome! Or you couldnt download the file from the Kaggle!!
## 目的是下载kaggle的blog二进制文件。很少有浏览器能支持大规模的blob传输。
## 只有上述浏览器才能成功下载保存的zip文件

# Before training, Do not forget turn on the settings of "INTERNET"! And Do not forget choose GPU as your accelerator!(Tesla T2 recommended)
# 在训练前注意打开internet,将Accelerator改成任意一个GPU(建议T4)
> You may need verify your phone number to be accessible to GPU if you are new Kaggle user.
对于新手玩家需要验证一下手机号才能使用GPU。

# 注意，目前只能使用训练webui，无法使用uvr5,和推理的webui。
### 解决方法:自己fork一下官方仓库，修改./GPT_SoVITS/inference_webui.py的gradio的share属性为true。同理uvr5也如此，将第一步的原仓库改成你的仓库地址。即可进行推理。输出gradio的public link

$ C_{n}^{m} = \frac{n!}{m!(n-m)!} $
$ \text{最诚挚的祝福} $

## 所有提示信息都写在每个代码框的注释里。

# 1.必要:安装环境(可替换你的fork地址)
# 1. Important: Install Dep(Always run it first whatever happened)

In [1]:
#准备环境,不管以前保没保存环境都必做.
%cd /kaggle/working
!git clone --depth=1 https://github.com/RVC-Boss/GPT-SoVITS.git
%cd GPT-SoVITS
!apt-get update && apt-get install -y --no-install-recommends tzdata ffmpeg libsox-dev parallel aria2 git git-lfs && git lfs install
!pip install -r requirements.txt

/kaggle/working
fatal: destination path 'GPT-SoVITS' already exists and is not an empty directory.
/kaggle/working/GPT-SoVITS
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:5 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]   
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1618 kB]
Get:7 https://packages.cloud.google.com/apt cloud-sdk InRelease [1616 B]       
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]      
Get:9 https://packages.cloud.google.com/apt google-fast-socket InRelease [1071 B]
Get:10 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [25.6 kB]
Get:11 http://security.ubun

# 2.下载预训练模型
# 2. Download pre-trained models for following traning.
### You dont have to run if you have saved your env(check it out through settings, see if it has 10G or more data )

In [2]:
#注意! 如果以前保存过环境就没必要进行这个代码框了!直接去下一个代码框!!!!
#具体点开你右上角菜单，看是否有10G以上的数据存放在里面。或者说在启动的时候加载很慢，提示
# Loading data之类的。
!mkdir -p /kaggle/working/GPT-SoVITS/GPT_SoVITS/pretrained_models
!mkdir -p /kaggle/working/GPT-SoVITS/tools/asr/models
!mkdir -p /kaggle/working/GPT-SoVITS/tools/uvr5
%cd /kaggle/working/GPT-SoVITS/GPT_SoVITS/pretrained_models
!git clone https://huggingface.co/lj1995/GPT-SoVITS
%cd /kaggle/working/GPT-SoVITS/tools/asr/models
!git clone https://www.modelscope.cn/damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch.git
!git clone https://www.modelscope.cn/damo/speech_fsmn_vad_zh-cn-16k-common-pytorch.git
!git clone https://www.modelscope.cn/damo/punc_ct-transformer_zh-cn-common-vocab272727-pytorch.git

# 这里注释掉uvr5模型是因为你用不上它。也是用不了。
# 除了开头的方式,如果你有更好的内网穿透工具并且知道如何引用，欢迎更新version。
# 来穿透uvr5 webui端口可以一试。

#%cd /kaggle/working/GPT-SoVITS/tools/uvr5
#!git clone https://huggingface.co/Delik/uvr5_weights
#!git config core.sparseCheckout true

!mv /kaggle/working/GPT-SoVITS/GPT_SoVITS/pretrained_models/GPT-SoVITS/* /kaggle/working/GPT-SoVITS/GPT_SoVITS/pretrained_models/

/kaggle/working/GPT-SoVITS/GPT_SoVITS/pretrained_models
fatal: destination path 'GPT-SoVITS' already exists and is not an empty directory.
/kaggle/working/GPT-SoVITS/tools/asr/models
fatal: destination path 'speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch' already exists and is not an empty directory.
fatal: destination path 'speech_fsmn_vad_zh-cn-16k-common-pytorch' already exists and is not an empty directory.
fatal: destination path 'punc_ct-transformer_zh-cn-common-vocab272727-pytorch' already exists and is not an empty directory.
mv: cannot stat '/kaggle/working/GPT-SoVITS/GPT_SoVITS/pretrained_models/GPT-SoVITS/*': No such file or directory


In [4]:
!cp /kaggle/input/neurosama/audio.m4s /kaggle/working/
!apt install -y ffmpeg
%cd /kaggle/working
!ffmpeg -i audio.m4s -vn -acodec pcm_s16le -ar 44100 -ac 2 neurosama.wav
%cd GPT-SoVITS
!rm -rf input/*

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 95 not upgraded.
/kaggle/working
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-

In [ ]:
%cd /kaggle/working/GPT-SoVITS
!mkdir -p input opt opt/slicer_opt opt/uvr5_opt opt/denoise_opt
!cp ../neuros

# Webui 启动!

In [5]:
#启动webui.py
# 通过npm插件启动单一端口的内网穿透。不能多开。
#  也就是只能训练，不能用来云端推理和进行uvr5的降噪

%cd /kaggle/working/GPT-SoVITS/
!npm install -g localtunnel
import subprocess
import threading
import time
import socket
import urllib.request
def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock= socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
        from colorama import Fore, Style
    print (Fore.GREEN + "\nIP: ", Fore. RED, urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"), "\n", Style. RESET_ALL)
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line.decode(), end='')
threading.Thread (target=iframe_thread, daemon=True, args=(9874,)).start()
!pip install gradio==3.38.0
!python  webui.py

/kaggle/working/GPT-SoVITS
m#################.] / reify:yargs-parser: http fetch GET 200 https://registry.
added 22 packages in 2s

3 packages are looking for funding
  run `npm fund` for details
npm notice 
npm notice New minor version of npm available! 10.1.0 -> 10.8.2
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.8.2
npm notice Run npm install -g npm@10.8.2 to update!
npm notice 
IMPORTANT: You are using gradio version 3.38.0, however version 4.29.0 is available, please upgrade.
--------
Running on local URL:  http://0.0.0.0:9874

IP:   35.185.238.103 
 
your url is: https://ten-pets-roll.loca.lt
^C
Keyboard interruption in main thread... closing server.


***执行下一步之前必须要停止webui进程才能继续!***

***When it comes to pack your weights,stop every running cell first.***

In [ ]:
#打包权重文件到/kaggle/working/GPT-SoVITS文件夹内，点击刷新按钮即可看到大包邮的内容.
%cd /kaggle/working/GPT-SoVITS/
!zip AnyName_GPT_packed.zip /kaggle/working/GPT-SoVITS/SoVITS_weights/* /kaggle/working/GPT-SoVITS/GPT_weights/TimingNorth*

In [ ]:
#清理GPU显存
#当你用到它的时候，大概率没什么用。肯定是webui填写的时候GPU卡号多写了一组，例如0-1,0-1。正确的应该是
# 0-1.也有可能batch size太高爆显存了，从7拉低到5即可完美解决。
!nvidia-smi
import numpy
import cuda
import torch
torch.cuda.empty_cache()

## 保持neat: 删除所有音频用来下一次训练
## Keep tidy and neat! the cell below will delete akl cache data for trainibg next character.

In [ ]:
# 完全重置，用于新的训练。
# Factory Reset GPT-SoVITS Dir For next training
%cd /kaggle/working/GPT-SoVITS
#这部分必须删除，否则会对新模型训练产生不可逆转的影响。
!rm -rf ./output/slicer_opt/*.wav #删除你的切片文件
!rm -rf ./output/denoise_opt/* #删除你降噪后的文件
!rm -rf ./logs/* # 删除log会清空训练过程中产生的数据。但不会清空结果生成的权重。
# 如果你不是想拿它继续训练，就可以删掉它。

!rm -rf ./*.zip

#注意，为了节省空间，建议将打包的zip文件执行删除!
#请在执行之前确保你已经下载完权重文件，为避免误操作，已经注释这个功能
#若不想删除，保持注释上面这一行即可。但是随着训练任务的增多
#会不知不觉地占用你的空间
!rm -rf ./output/asr_opt/* #删除生成的文本文件
!rm -rf ./input/* #删除你的样本未处理文件

#为避免误操作，以下为注释内容。
!rm -rf ./GPT_weights/*
!rm -rf ./SoVITS_weights/*

In [ ]:
!cp /kaggle/input/timing/* /kaggle/working/GPT-SoVITS/input -r